In [61]:
import xml.etree.ElementTree as ET
from Components import Timing, Course, Subpart, Class, HardConstraint, SoftConstraint, Student

inputFile = "bet-sum18_trimmed.xml"

In [ ]:
# Passing the path of the 
# xml document to enable the 
# parsing process 
tree = ET.parse(inputFile) 
# getting the parent tag of 
# the xml document 
root = tree.getroot() 

In [52]:
coursesXML = root.findall('./courses/course')
ET.tostringlist

<function xml.etree.ElementTree.tostringlist(element, encoding=None, method=None, *, xml_declaration=None, default_namespace=None, short_empty_elements=True)>

In [53]:
courses = []
subparts = []
classes = []
for course in coursesXML:
    config = course.find('config')
    courseID = course.get('id')
    subpartXML = config.findall('subpart')
    subpartList = []
    for subpart in subpartXML:
        subpartID = subpart.get('id')
        subpartList.append(subpartID)
        classesXML = subpart.findall('class')
        classList = []
        for singleClass in classesXML:
            classID = singleClass.get('id')
            classList.append(classID)
            classLimit = singleClass.get('limit')
            classParent = singleClass.get('parent')
            timeXML = singleClass.findall('time')
            timings = []
            for timing in timeXML:
                days = timing.get('days')
                length = timing.get('length')
                penalty = timing.get('penalty')
                start = timing.get('start')
                weeks = timing.get('weeks')
                timingObject = Timing(days, length, start, weeks, penalty)
                timings.append(timingObject)
            classObject = Class(classID, classLimit, classParent, subpartID, courseID, timings)
            classes.append(classObject)
        subpartObject = Subpart(subpartID, courseID, classList)
        subparts.append(subpartObject)
    courseObject = Course(courseID, subpartList)
    courses.append(courseObject)
print("%d Classes extracted"%(len(classes)))
print("%d Subparts extracted"%(len(subparts)))
print("%d Courses extracted"%(len(courses)))

99 Classes extracted
71 Subparts extracted
48 Courses extracted


In [66]:
Hardconstraints = []
SoftConstraints = []
distributionXML = root.findall('./distributions/distribution')

requiredTypes = ['Precedence', 'NotOverlap', 'SameTime', 'DifferentTime', 'MinGap', 'DifferentDays']

for dist in distributionXML:
    constraintType = dist.get('type')
    for checkstring in requiredTypes:
        if checkstring in constraintType:
            classList = []
            classXML = dist.findall('class')
            for singleClass in classXML:
                classList.append(singleClass.get('id'))
            if(dist.get('required') == 'true'):
                Hardconstraints.append(HardConstraint(constraintType, classList))
            else:
                penalty = dist.get('penalty')
                SoftConstraints.append(SoftConstraint(constraintType, classList, penalty))